In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
df_train = pd.read_pickle("intermediate/df_train.pkl") 

# Delete noisy images from training set

In [ ]:
# function to remove black / dark borders from image
# threshold = pixel intensity threshold to consider a pixel as non-black

def auto_crop_black_borders(img, threshold=10):
    
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img

    # Create a binary mask of non-black pixels
    mask = gray > threshold

    # Find the bounding box of the non-black area
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)

    if not np.any(rows) or not np.any(cols):
        return img  # nothing to crop

    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]

    cropped = img[y_min:y_max+1, x_min:x_max+1]
    return cropped

In [ ]:
# function to identify images dominated by one color -> remove bad DQ cases

def split_by_color_dominance(df, path_col, threshold=0.25):
    good, bad = [], []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        path = row[path_col]
        img = cv2.imread(path)
        if img is None:
            continue

        img = auto_crop_black_borders(img)
        b, g, r = cv2.split(img)
        total = b.astype(np.float32) + g + r + 1e-5
        ratios = [np.mean(c / total) for c in (r, g, b)]

        (good if min(ratios) > threshold else bad).append(row)

    return pd.DataFrame(good), pd.DataFrame(bad)

In [ ]:
# split up train data into clean and unclean images

df_train_clean, df_train_unclean = split_by_color_dominance(df_train, 'full_path')

In [ ]:
df_train_clean.shape

In [ ]:
df_train_unclean

In [ ]:
img = cv2.imread("images/italy/1741644879_46.4328734_13.6278.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
img = auto_crop_black_borders(img)

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
df_train_clean = df_train_clean.reset_index()
df_train_clean.tail

In [ ]:
df_train_clean.to_pickle("intermediate/df_train_clean.pkl") 
df_train_unclean.to_pickle("intermediate/df_train_unclean.pkl") 

In [ ]:
df_train_clean = pd.read_pickle("intermediate/df_train_clean.pkl") 

# Slice training images into square images

In [ ]:
# function to convert equirectangular ('panoramic') image to perspective ('normal') image

# fov = horizontal field of view (degrees)
# theta = rotation angle (degrees)
# size = output image size

def equirectangular_to_perspective(equi_img, fov, theta, size=512):

    height, width = size, size
    equ_h, equ_w = equi_img.shape[:2]

    # Convert angles to radians
    fov_rad = np.deg2rad(fov)
    theta_rad = np.deg2rad(theta)

    # Grid of x, y in normalized view space
    x = np.linspace(-np.tan(fov_rad / 2), np.tan(fov_rad / 2), width)
    y = np.linspace(-1, 1, height)  # keep vertical stretch simple
    x, y = np.meshgrid(x, -y)  # flip y for image orientation
    z = np.ones_like(x)

    # Normalize direction vectors
    norm = np.sqrt(x**2 + y**2 + z**2)
    x /= norm
    y /= norm
    z /= norm

    # Rotate around Y axis (theta)
    x_rot = np.cos(theta_rad) * x + np.sin(theta_rad) * z
    z_rot = -np.sin(theta_rad) * x + np.cos(theta_rad) * z

    # Convert to spherical coordinates
    lon = np.arctan2(x_rot, z_rot)
    lat = np.arcsin(y)

    # Map to image coordinates
    u = (lon + np.pi) / (2 * np.pi) * equ_w
    v = (np.pi / 2 - lat) / np.pi * equ_h

    # Remap
    u = u.astype(np.float32)
    v = v.astype(np.float32)
    perspective = cv2.remap(equi_img, u, v, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)

    return perspective

In [ ]:
# function that first removes black borders, then converts it to 'normal' perspective, then slices it into 4 images and saves them

def slice_square_images(image_path, size = 512):
    img = cv2.imread(image_path)
    img = auto_crop_black_borders(img)  # Remove black borders if needed

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    views = ['front', 'right', 'back', 'left']

    image_name = image_path.split('/')[2].replace('.jpg', '')
    country = image_path.split('/')[1]
    
    output_folder = 'train_clean_images_square/' + country + '/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i in range(4):  
        view = equirectangular_to_perspective(img, fov=90, theta=i*90, size = size)

        plt.imshow(view)
        plt.axis('off')
        plt.show()
        
        new_image_path = os.path.join(output_folder + f"{image_name}_{views[i]}.jpg")
        cv2.imwrite(new_image_path, view)

In [ ]:
slice_square_images('images/japan/1741695546_36.823432_139.5921591.jpg')

In [ ]:
# slice all training images into 4 square images

tqdm.pandas() # to see a progress bar

df_train_clean['full_path'].progress_apply(slice_square_images)

# Make new df with paths of all square images

In [ ]:
def get_paths_of_square_images(path):
    new_paths = []
    views = ['front', 'right', 'back', 'left']
    parts = path.replace(".jpg", "").split("/")
    for view in views:
        path = 'train_clean_images_square/' + f"{parts[1]}/{parts[2]}_{view}.jpg"
        new_paths.append(path)
    return new_paths

In [ ]:
# apply the function to 'path' column and create a new column 'new_paths'

df_train_clean['sq_image_path'] = df_train_clean['full_path'].apply(get_paths_of_square_images)

In [ ]:
# explode the new column into multiple rows

df_train_clean_sq = df_train_clean.explode('sq_image_path', ignore_index=True).copy()

df_train_clean_sq.drop(['level_0', 'filename', 'color', 'set'], axis = 1, inplace = True) # remove irrelevant columns

In [ ]:
# convert region cluster to str variable to use as classification target variable

df_train_clean_sq['region_cluster_str'] = "region_" + df_train_clean_sq['region_cluster'].astype(str)

In [ ]:
df_train_clean_sq.tail(10)

In [ ]:
df_train_clean_sq.shape

# Finetune model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# make train and validation set, making sure that slices from the same original image are kept together

from sklearn.model_selection import GroupShuffleSplit

splitter = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=16)
train_idx, val_idx = next(splitter.split(df_train_clean_sq, groups=df_train_clean_sq['full_path']))

train_df_gen = df_train_clean_sq.iloc[train_idx]
val_df_gen = df_train_clean_sq.iloc[val_idx]

In [ ]:
# create separate train and validation set + normalize pixel values

train_val_datagen = ImageDataGenerator(rescale=1/255) 

In [ ]:
train_generator = train_val_datagen.flow_from_dataframe(
    dataframe= train_df_gen,  
    x_col='sq_image_path',
    y_col='country',
    #y_col = 'region_cluster_str',
    target_size=(224, 224),  # resize images to fit the model
    batch_size=16,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
# note: slices from the same picture will be spread across train and val!

validation_generator = train_val_datagen.flow_from_dataframe(
    dataframe= val_df_gen,  
    x_col='sq_image_path',
    y_col='country',
    #y_col = 'region_cluster_str',
    target_size=(224, 224),  # resize images to fit the model
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# load EfficientNetB0 without the top layer (classification layer)

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# freeze the base model layers

base_model.trainable = False

In [ ]:
# create sequential model

model_finetune = Sequential()

In [ ]:
# define data augmentations that are relevant (realistic) for these images

data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),   # Flip images horizontally
    layers.RandomBrightness(0.2),      # Randomly adjust the brightness by up to 20%
    layers.RandomContrast(0.1),
    layers.RandomZoom(0.2)
])

In [ ]:
# add the data augmentation layers

model_finetune.add(data_augmentation)

In [ ]:
# add the frozen base model

model_finetune.add(base_model)

In [ ]:
# add a global average pooling layer to reduce dimensionality

model_finetune.add(GlobalAveragePooling2D())

In [ ]:
# add dense layers, include dropout layers to avoid overfitting

model_finetune.add(Dense(256, activation='relu'))
model_finetune.add(Dropout(0.4))
model_finetune.add(Dense(64, activation='relu'))
model_finetune.add(Dropout(0.3))

In [ ]:
# add a final dense layer used for prediction

model_finetune.add(Dense(df_train_clean_sq['country'].nunique(), activation='softmax')) 
#model_finetune.add(Dense(df_train_clean_sq['region_cluster_str'].nunique(), activation='softmax')) 

In [ ]:
model_finetune.summary()

In [ ]:
print(repr(model_finetune))

In [ ]:
model_finetune.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) # set learning rate less small?

In [ ]:
history = model_finetune.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),  # Stops early if validation performance doesn't improve
        ModelCheckpoint('best_model_2.keras', save_best_only=True)  # Save the best model during training
    ]
)

In [ ]:
# fine-tune the model: unfreeze some layers

base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False  # freeze the first 100 layers

In [ ]:
model_finetune.summary()

In [ ]:
model_finetune.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy']) # set learning rate less small?

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model_6.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = model_finetune.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, checkpoint],
    verbose=1  # Optional: shows epoch progress
)

In [ ]:
print(train_df_gen['region_cluster_str'].value_counts(normalize=True))

print(val_df_gen['region_cluster_str'].value_counts(normalize=True))

In [ ]:
print(train_df_gen['country'].value_counts(normalize=True))

print(val_df_gen['country'].value_counts(normalize=True))